In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot

from Recommenders.Similarity.Compute_Similarity import SimilarityFunction
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Utils.DataReader import load_urm, load_icm,
from operator import itemgetter

In [3]:
URM_all = load_urm()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.85)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 4337 (10.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 3128 ( 7.5%) Users that have less than 1 test interactions


In [4]:
ICM_all = load_icm()
ICM_all

<27968x8 sparse matrix of type '<class 'numpy.intc'>'
	with 46181 stored elements in Compressed Sparse Row format>

In [5]:
MAP_CBF=[]
MAP_CF=[]

for similarity in ['cosine','pearson','jaccard','tanimoto','adjusted']:

    for topK in [10, 50, 100, 200, 500]:
        for shrink in [0, 10, 50, 100, 200, 500]:
            recommender = ItemKNNCFRecommender(URM_train)
            recommender.fit(topK=topK, shrink=shrink, similarity=similarity, normalize=False)

            result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

            MAP_CF.append(('similarity={}'.format(similarity),'topK={}'.format(topK),'shrink={}'.format(shrink),result_dict[10]["MAP"]))

            print(MAP_CF[-1])

    for topK in [10, 50, 100, 200, 500]:
        for shrink in [0, 10, 50, 100, 200, 500]:
            recommender = ItemKNNCBFRecommender(URM_train=URM_train, ICM_train=ICM_all)
            recommender.fit(topK=topK, shrink=shrink, similarity=similarity, normalize=False)

            result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

            MAP_CBF.append(('similarity={}'.format(similarity),'topK={}'.format(topK),'shrink={}'.format(shrink),result_dict[10]["MAP"]))
            print(MAP_CBF[-1])

ItemKNNCFRecommender: URM Detected 3 ( 0.0%) users with no interactions.
Similarity column 24507 (100.0%), 5450.11 column/sec. Elapsed time 4.50 sec
EvaluatorHoldout: Processed 37292 (100.0%) in 46.49 sec. Users per second: 802


KeyError: 10